<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/fittedbonddiscountcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
class FittedBondDiscountCurveTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Default eval date for tests, can be overridden locally
        ql.Settings.instance().evaluationDate = ql.Date(15, ql.May, 2020)

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testEvaluation(self):
        print("Testing that fitted bond curves work as evaluators...")
        today = ql.Settings.instance().evaluationDate

        bond = ql.ZeroCouponBond(3, ql.TARGET(), 100.0, today + ql.Period(10, ql.Years))
        q_handle = make_quote_handle(100.0)

        helpers = [ql.BondHelper(q_handle, bond)]

        # ExponentialSplinesFitting()
        fitting_method = ql.ExponentialSplinesFitting()

        maxIterations = 0 # No iterations, just use the guess
        guess_list = [-51293.44, -212240.36, 168668.51, 88792.74,
                      120712.13, -34332.83, -66479.66, 13605.17, 0.0]
        guess_arr = ql.Array(guess_list)

        # FittedBondDiscountCurve(settlementDays, calendar, bondHelpers,
        #                         dayCounter, fittingMethod, accuracy, maxIterations,
        #                         guessArray, minCutoffTime, maxCutoffTime)
        # The C++ constructor takes (referenceDate, ...) or (settlementDays, ...)
        # Python: FittedBondDiscountCurve(referenceDate, bondHelpers, dayCounter, fittingMethod,
        #                                  accuracy=1.0e-10, maxIterations=10000, guess=Array(),
        #                                  minCutoffTime=0.0, maxCutoffTime=QL_MAX_REAL)
        # Or (settlement_days, calendar, ...)

        # Using the constructor that takes referenceDate
        curve = ql.FittedBondDiscountCurve(today, helpers, ql.Actual365Fixed(),
                                          fitting_method, 1e-10, maxIterations, guess_arr)

        # Check if discount can be called without throwing
        try:
            df = curve.discount(3.0)
            self.assertTrue(isinstance(df, float)) # Basic check that it returns a float
        except Exception as e:
            self.fail(f"curve.discount(3.0) threw an exception: {e}")


    def testFlatExtrapolation(self):
        print("Testing fitted bond curve with flat extrapolation...")
        asof_date = ql.Date(15, ql.July, 2019)
        ql.Settings.instance().evaluationDate = asof_date

        quotes_data = [101.2100, 100.6270, 99.9210, 101.6700]
        bonds = []
        calendar = ql.Canada()
        dc_bond = ql.ActualActual(ql.ActualActual.ISDA) # Day counter for bonds

        # Bond definitions (matching C++ exactly)
        # EJ5346956
        bonds.append(ql.FixedRateBond(
            2, 100.0, # settlementDays, faceAmount
            ql.Schedule(ql.Date(1, ql.February, 2013), ql.Date(3, ql.February, 2020),
                        ql.Period(6, ql.Months), calendar, ql.Following, ql.Following,
                        ql.DateGeneration.Forward, False, ql.Date(3, ql.August, 2013)),
            [0.046], dc_bond # coupon rates
        ))
        # EK9689119
        bonds.append(ql.FixedRateBond(
            2, 100.0,
            ql.Schedule(ql.Date(12, ql.June, 2015), ql.Date(12, ql.June, 2020),
                        ql.Period(6, ql.Months), calendar, ql.Following, ql.Following,
                        ql.DateGeneration.Forward, False, ql.Date(12, ql.December, 2015)),
            [0.0295], dc_bond
        ))
        # AQ1410069
        bonds.append(ql.FixedRateBond(
            2, 100.0,
            ql.Schedule(ql.Date(24, ql.November, 2017), ql.Date(24, ql.November, 2020),
                        ql.Period(6, ql.Months), calendar, ql.Following, ql.Following,
                        ql.DateGeneration.Forward, False, ql.Date(24, ql.May, 2018)),
            [0.02689], dc_bond
        ))
        # AM5387676
        bonds.append(ql.FixedRateBond(
            2, 100.0,
            ql.Schedule(ql.Date(21, ql.February, 2017), ql.Date(21, ql.February, 2022),
                        ql.Period(6, ql.Months), calendar, ql.Following, ql.Following,
                        ql.DateGeneration.Forward, False, ql.Date(21, ql.August, 2017)),
            [0.0338], dc_bond
        ))

        helpers = [ql.BondHelper(make_quote_handle(quotes_data[i]), bonds[i]) for i in range(len(bonds))]

        dc_curve = ql.Actual365Fixed()

        # Method1: NelsonSiegelFitting with default extrapolation
        method1 = ql.NelsonSiegelFitting()

        # Method2: NelsonSiegelFitting with flat extrapolation cutoffs
        # minCutoffTime = dc_curve.yearFraction(asof_date, helpers[0].bond().maturityDate())
        # maxCutoffTime = dc_curve.yearFraction(asof_date, helpers[-1].bond().maturityDate())
        # NelsonSiegelFitting(weights, optimizationMethod, l2penalty,
        #                     minCutoffTime, maxCutoffTime)
        min_cutoff_time = dc_curve.yearFraction(asof_date, bonds[0].maturityDate())
        max_cutoff_time = dc_curve.yearFraction(asof_date, bonds[-1].maturityDate())
        method2 = ql.NelsonSiegelFitting(ql.Array(), None, ql.Array(),
                                         min_cutoff_time, max_cutoff_time)

        guess_list = [0.0317, 5.0, -3.6796, 24.1703]
        guess_arr = ql.Array(guess_list)

        curve1 = ql.FittedBondDiscountCurve(asof_date, helpers, dc_curve, method1,
                                            1e-10, 10000, guess_arr)
        curve2 = ql.FittedBondDiscountCurve(asof_date, helpers, dc_curve, method2,
                                            1e-10, 10000, guess_arr)

        curve1.enableExtrapolation()
        curve2.enableExtrapolation()

        engine1 = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(curve1))
        engine2 = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(curve2))

        modelPrices1_clean = []
        modelPrices2_clean = []
        modelYields2 = [] # For curveYield2 comparison

        for i, bond_obj in enumerate(bonds):
            bond_obj.setPricingEngine(engine1)
            modelPrices1_clean.append(bond_obj.cleanPrice())

            bond_obj.setPricingEngine(engine2)
            modelPrices2_clean.append(bond_obj.cleanPrice())
            # For QL_CHECK_CLOSE later, store yield from model2 pricing
            modelYields2.append(bond_obj.yieldRate(modelPrices2_clean[-1], dc_curve, ql.Continuous, ql.NoFrequency))


        self.assertEqual(curve1.fitResults().errorCode(), ql.EndCriteria.MaxIterations)
        self.assertEqual(curve2.fitResults().errorCode(), ql.EndCriteria.MaxIterations)

        # C++: QL_CHECK_CLOSE(modelYield2, curveYield2, 1.0); // 1.0 percent relative tolerance
        # which means abs(a-b) <= 1.0 * abs(b) / 100 or abs(a-b) <= 1.0 * abs(a) / 100
        # In Python, assertAlmostEqual uses delta. We need to calculate the appropriate delta.
        # For relative tolerance of 1.0 (meaning 100% relative diff, or 1.0 absolute if values are small)
        # The comment "1.0 percent relative tolerance" implies N=1.0 for QL_CHECK_CLOSE.
        # N=1.0 means relative tolerance of 1.0% == 0.01.

        for i in range(len(helpers)):
            t_maturity = curve1.timeFromReference(bonds[i].maturityDate())

            # curveYield1 = curve1.zeroRate(t_maturity, ql.Continuous, ql.NoFrequency).rate()
            # Python: curve1.zeroRate(t_maturity, ql.Continuous).rate() or zeroRate(Date, ...)
            curveYield1 = curve1.zeroRate(t_maturity, ql.Continuous, ql.NoFrequency, True).rate()
            curveYield2 = curve2.zeroRate(t_maturity, ql.Continuous, ql.NoFrequency, True).rate()

            # "expecting huge yield" for curve1
            self.assertGreater(curveYield1, 1.0,
                               f"Curve1 yield {curveYield1} not > 1.0 as expected for bond {i}")

            # QL_CHECK_CLOSE(modelYield2, curveYield2, 1.0);
            # 1.0 here is N, meaning a relative tolerance of N% = 1.0% = 0.01
            # self.assertAlmostEqual(a, b, delta = abs(b) * 0.01) or similar
            # For simplicity, use a direct relative check.
            relative_diff_yield2 = abs(modelYields2[i] - curveYield2) / abs(curveYield2 if curveYield2 != 0 else 1.0)
            self.assertLessEqual(relative_diff_yield2, 0.01, # 1% relative tolerance
                                 f"ModelYield2 vs CurveYield2 mismatch for bond {i}: "
                                 f"{modelYields2[i]:.6f} vs {curveYield2:.6f}")

        # Resetting guess for curve1
        # C++: curve1->resetGuess({ 0.02, 0.0, 0.0, 0.0 });
        # This implies a call to the fitting method's setSolution or similar, then re-performFit.
        # FittedBondDiscountCurve doesn't have a direct resetGuess method in Python.
        # We might need to re-create the curve with the new guess, or see if the fitting_method can be updated.
        # The C++ might have a method to update the guess and re-trigger fitting.
        # In Python, the fitting happens at construction. If `fittingMethod` itself holds the guess
        # and can be refit, that's one way. Or reconstruct.
        # Let's assume we reconstruct curve1 with the new guess.

        new_guess_list = [0.02, 0.0, 0.0, 0.0]
        new_guess_arr = ql.Array(new_guess_list)
        # Reconstruct curve1 (or if there's a refit method, use that)
        # NelsonSiegelFitting has a setSolution method that could be used if we
        # want to change the *internal* guess before a fit, but FittedBondDiscountCurve
        # takes the guess at construction. The C++ `resetGuess` might be a custom method.
        # For this test, let's assume `resetGuess` means re-fitting with a new initial guess.
        # This is best done by creating a new curve.

        # To strictly follow C++ `resetGuess` if it forces a refit with a new *initial* guess:
        # The `FittingMethod` object (method1) is shared. If `resetGuess` modified method1's internal
        # state before refitting, that's complex.
        # However, FittedBondDiscountCurve takes the guess array directly.
        # The C++ `resetGuess` is likely on the `FittedBondDiscountCurve` itself,
        # implying it re-runs its internal fitting procedure.
        # Python's `FittedBondDiscountCurve` does not seem to have `resetGuess` or `performCalculations`
        # exposed in a way that changing the guess would trigger a refit post-construction easily.
        # The fit is done in the constructor.
        # So, to "reset guess" and refit, we must create a new curve.

        curve1_refit = ql.FittedBondDiscountCurve(asof_date, helpers, dc_curve, method1,
                                                 1e-10, 10000, new_guess_arr)
        curve1_refit.enableExtrapolation()

        # Pricing engine with refitted curve
        engine1_refit = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(curve1_refit))

        self.assertEqual(curve1_refit.fitResults().errorCode(), ql.EndCriteria.StationaryPoint)

        for i, bond_obj in enumerate(bonds):
            bond_obj.setPricingEngine(engine1_refit) # Use refitted engine
            # modelPrices1_clean were calculated with the *original* curve1.
            # We need to re-calculate model prices with curve1_refit to get modelYield1 for this check.
            current_model_price1_refit = bond_obj.cleanPrice()
            modelYield1_refit = bond_obj.yieldRate(current_model_price1_refit, dc_curve, ql.Continuous, ql.NoFrequency)

            t_maturity = curve1_refit.timeFromReference(bonds[i].maturityDate())
            curveYield1_refit = curve1_refit.zeroRate(t_maturity, ql.Continuous, ql.NoFrequency, True).rate()

            # QL_CHECK_CLOSE(modelYield1, curveYield1, 6); // N=6 => 6% relative tolerance
            relative_diff_yield1_refit = abs(modelYield1_refit - curveYield1_refit) / \
                                         abs(curveYield1_refit if curveYield1_refit != 0 else 1.0)
            self.assertLessEqual(relative_diff_yield1_refit, 0.06, # 6% relative tolerance
                                 f"Refit ModelYield1 vs CurveYield1 mismatch for bond {i}: "
                                 f"{modelYield1_refit:.6f} vs {curveYield1_refit:.6f}")


    def testRequiredGuess(self):
        print("Testing that fitted bond curves require a guess when given an L2 penalty...")
        today = ql.Settings.instance().evaluationDate
        bonds_defs = [
            (today + ql.Period(1, ql.Years), 99.0),
            (today + ql.Period(2, ql.Years), 98.0),
            (today + ql.Period(5, ql.Years), 95.0),
            (today + ql.Period(10, ql.Years), 90.0)
        ]
        helpers = []
        for mat_date, quote_val in bonds_defs:
            bond = ql.ZeroCouponBond(3, ql.TARGET(), 100.0, mat_date)
            helpers.append(ql.BondHelper(make_quote_handle(quote_val), bond))

        # NelsonSiegelFitting(weights, optimizationMethod, l2penalty)
        l2_penalty = ql.Array([0.25, 0.25, 0.25, 0.25])
        fitting_method_l2 = ql.NelsonSiegelFitting(ql.Array(), None, l2_penalty)

        # Construct without guess
        with self.assertRaisesRegex(RuntimeError, "L2 penalty requires a guess"):
            curve = ql.FittedBondDiscountCurve(today, helpers, ql.Actual365Fixed(),
                                              fitting_method_l2, 1e-10, 10000)
            # Accessing a method that triggers calculation if lazy
            curve.discount(3.0)


    def testGuessSize(self):
        print("Testing that fitted bond curves check the guess size when given...")
        today = ql.Settings.instance().evaluationDate
        # Same bond setup as testRequiredGuess
        bonds_defs = [
            (today + ql.Period(1, ql.Years), 99.0),
            (today + ql.Period(2, ql.Years), 98.0),
            (today + ql.Period(5, ql.Years), 95.0),
            (today + ql.Period(10, ql.Years), 90.0)
        ]
        helpers = []
        for mat_date, quote_val in bonds_defs:
            bond = ql.ZeroCouponBond(3, ql.TARGET(), 100.0, mat_date)
            helpers.append(ql.BondHelper(make_quote_handle(quote_val), bond))

        fitting_method_ns = ql.NelsonSiegelFitting() # Nelson-Siegel has 4 parameters

        guess_short_list = [0.01, 0.0, 0.0] # Too few for Nelson-Siegel
        guess_short_arr = ql.Array(guess_short_list)

        with self.assertRaisesRegex(RuntimeError, "wrong size for guess"):
            curve = ql.FittedBondDiscountCurve(today, helpers, ql.Actual365Fixed(),
                                              fitting_method_ns, 1e-10, 10000, guess_short_arr)
            curve.discount(3.0)


    def testConstraint(self):
        print("Testing that fitted bond curves respect passed constraint...")
        today = ql.Settings.instance().evaluationDate

        bond1 = ql.ZeroCouponBond(3, ql.TARGET(), 100.0, today + ql.Period(1, ql.Years))
        bond2 = ql.ZeroCouponBond(3, ql.TARGET(), 100.0, today + ql.Period(2, ql.Years))
        helpers = [
            ql.BondHelper(make_quote_handle(101.0), bond1), # Prices imply negative rates
            ql.BondHelper(make_quote_handle(102.0), bond2)
        ]

        accuracy = 1e-10
        maxIterations = 10000
        guess_arr_flat = ql.Array([0.01]) # Guess for PyFlatZero (1 parameter)

        # Unconstrained
        unconstrained_method = PyFlatZero() # Default is NoConstraint
        unconstrained_curve = ql.FittedBondDiscountCurve(
            today, helpers, ql.Actual365Fixed(), unconstrained_method,
            accuracy, maxIterations, guess_arr_flat)
        # Expect the fitted zero rate (solution[0]) to be negative
        self.assertLess(unconstrained_curve.fitResults().solution()[0], 0.0)

        # Constrained to be positive
        positive_constraint = ql.PositiveConstraint()
        positive_method = PyFlatZero(positive_constraint)
        positive_curve = ql.FittedBondDiscountCurve(
            today, helpers, ql.Actual365Fixed(), positive_method,
            accuracy, maxIterations, guess_arr_flat)
        # Expect the fitted zero rate to be positive (or zero if that's the boundary)
        self.assertGreaterEqual(positive_curve.fitResults().solution()[0], 0.0) # Should be > 0 due to constraint


if __name__ == '__main__':
    print("C++ test suite: FittedBondDiscountCurveTests")
    print("Python QuantLib version: " + ql.__version__)
    print("-------------------------------------------------------")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(FittedBondDiscountCurveTests))
    unittest.TextTestRunner(verbosity=2).run(suite)

Use code with caution.
Python
Key Considerations and Changes:
PyFlatZero Fitting Method:
The C++ test defines a custom FlatZero fitting method. I've created a Python equivalent PyFlatZero inheriting from ql.FittedBondDiscountCurveFittingMethod.
The clone() method from C++'s FittingMethod interface is not automatically part of Python's custom class unless explicitly required by SWIG for certain internal operations. I've commented it out for now, assuming direct use in Python doesn't need it. If QuantLib's internals try to clone it and fail, this would need to be implemented.
The PyFlatZero constructor now correctly passes the constraint argument to the base class ql.FittedBondDiscountCurveFittingMethod.
FittedBondDiscountCurve Constructor:
The Python constructor signatures can vary (e.g., taking referenceDate or settlementDays, calendar). I've used the referenceDate version primarily.
testFlatExtrapolation - resetGuess:
The C++ code has curve1->resetGuess(...). Python's FittedBondDiscountCurve doesn't expose a direct resetGuess method that refits with a new initial guess post-construction. The fitting is done in the constructor.
To simulate this, I've created a new FittedBondDiscountCurve instance (curve1_refit) with the new guess. This is the most straightforward way to achieve the same effect in Python if the original object cannot be internally refit with a new starting guess.
The model prices (modelPrices1_clean) and yields need to be recalculated using this curve1_refit for the second part of the assertion.
QL_CHECK_CLOSE:
The C++ macro QL_CHECK_CLOSE(a, b, N) checks if std::abs(a-b) <= N * std::abs(a)/100 (approximately, it's a relative tolerance of N%).
In Python, self.assertAlmostEqual(a, b, delta=...) is used. For relative tolerance, the delta needs to be calculated based on the expected value and the desired relative tolerance (e.g., delta = abs(expected_value) * relative_tol_percentage / 100.0). I've implemented direct relative difference checks where QL_CHECK_CLOSE was used with a percentage.
Error Handling:
BOOST_CHECK_EXCEPTION with ExpectedErrorMessage is translated to self.assertRaisesRegex(ExceptionType, "regex_for_message").
Bond Definitions: The bond constructions are replicated as closely as possible, including schedules, day counters, and coupon rates.
Yield Calculation: bond.yieldRate(...) is used in Python where C++ used bond->yield(...). Parameters for compounding and frequency are matched.
Dates and Settings: ql.Settings.instance().evaluationDate is set at the beginning of tests or setUp for reproducibility.
std::make_shared and ext::make_shared: These C++ smart pointer creations map to direct object instantiation in Python (e.g., ql.ZeroCouponBond(...)).
Handle<Quote>: make_quote_handle helper is used for consistency.
Array() vs None for Optional Args: When a C++ function takes an optional Array (like NelsonSiegelFitting for weights or l2penalty), passing an empty ql.Array() or None (if the binding supports it) is the Python equivalent. I've used ql.Array() for empty arrays and None for optional OptimizationMethod pointers.